# Importaciones

In [ ]:
import sys
!{sys.executable} -m pip install tensorflow gdown --quiet

In [ ]:
import os
import zipfile
import gdown
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import files
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import load_model

# Creación de estructura y Particiones

In [ ]:
zip_path = './data/dataset.zip'
url = 'https://drive.google.com/uc?id=1vOafMTcq3i_SzexClnkEfv0HfLPuh-Zz'

# Crear carpeta data
os.makedirs('./data', exist_ok=True)

# Descargar el dataset
gdown.download(url, zip_path, quiet=False)
print("Descarga completada, descomprimiendo...")

# Descomprimir
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('./data/')
os.remove(zip_path)  # Borrar el zip después de extraer
print("Dataset descomprimido correctamente.")

In [ ]:
ruta_imagenes = "./data/Sign Language Dataset"

# Lista donde almacenaremos la ruta de cada imagen y su etiqueta
datos = []
# Extensiones válidas para considerar un archivo como imagen
extensiones_permitidas = ('.jpg', '.jpeg', '.png', '.bmp')

# Recorremos cada subcarpeta dentro de la carpeta principal (cada subcarpeta es una clase)
for clase in os.listdir(ruta_imagenes):
    ruta_clase = os.path.join(ruta_imagenes, clase) # Ruta completa a la carpeta de la clase
    if os.path.isdir(ruta_clase):
        # Recorremos los archivos dentro de esa clase
        for archivo in os.listdir(ruta_clase):
            # Verificamos que el archivo sea una imagen
            if archivo.lower().endswith(extensiones_permitidas):
                # Añadimos un diccionario con la ruta completa y su etiqueta
                datos.append({
                    'filename': os.path.join(ruta_clase, archivo),
                    'class': clase
                })

# Si no se encontraron imágenes, mostramos un error
if not datos:
    raise ValueError("No se encontraron imágenes en el dataset. Verifica la ruta.")

In [ ]:
# Convertir a DataFrame
df = pd.DataFrame(datos)

# Dividir en train, val y test
df_train_val, df_test = train_test_split(df, test_size=0.05, stratify=df['class'], random_state=42)
df_train, df_val = train_test_split(df_train_val, test_size=0.20, stratify=df_train_val['class'], random_state=42)

# Verificación del tamaño de las divisiones
print(f'Tamaño entrenamiento: {len(df_train)}')
print(f'Tamaño validación: {len(df_val)}')
print(f'Tamaño test: {len(df_test)}')

#Crear generadores

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)    # Normalizar imágenes al rango [0,1]

train_generator = datagen.flow_from_dataframe(
    df_train,
    x_col='filename',
    y_col='class',
    target_size=(100, 100),   # Redimensionar todas las imágenes a 100x100
    color_mode='grayscale',   # Convertir a escala de grises (1 canal)
    batch_size=20,
    class_mode='categorical',   # Etiquetas one-hot para clasificación multiclase
    shuffle=True,   # Barajar imágenes para entrenamiento
    seed=42   # Semilla para reproducibilidad
)

val_generator = datagen.flow_from_dataframe(
    df_val,
    x_col='filename',
    y_col='class',
    target_size=(100, 100),
    color_mode='grayscale',
    batch_size=20,
    class_mode='categorical',
    shuffle=False   # Mantener el orden fijo para validación
)

test_generator = datagen.flow_from_dataframe(
    df_test,
    x_col='filename',
    y_col='class',
    target_size=(100, 100),
    color_mode='grayscale',
    batch_size=20,
    class_mode='categorical',
    shuffle=False   # Mantener el orden fijo para test
)

# Mostrar ejemplo

In [ ]:
# Obtener un lote del generador
x_batch, _ = next(train_generator)  # Ignoramos las etiquetas

# Tomar la primera imagen del batch
img = x_batch[0]  # Imagen redimensionada y normalizada

# Mostrar la imagen
plt.imshow(img.squeeze(), cmap='gray')  # squeeze para quitar el canal extra
plt.axis('off')
plt.show()

In [ ]:
print(img.squeeze())  # Mostrar la matriz 100x100

# Creación de la Primera Red Neuronal

La Red Neuronal contará con dos capas convolucionales y una capa densa.

In [ ]:
model = Sequential()

# Primera capa convolucional
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(100,100,1))) # 32 filtros, kernel de 3x3, función activación relu, 100x100 pixeles, 1 Canal
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))

# Segunda capa convolucional
model.add(Conv2D(64, (3,3), activation='relu')) # 64 filtros
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))

# Aplanar para pasar a la capa densa
model.add(Flatten())

# Capa densa totalmente conectada
model.add(Dense(128, activation='relu')) # 128 neuronas
model.add(Dropout(0.5))  # Evitar overfitting

# Capa de salida (10 clases)
model.add(Dense(10, activation='softmax')) # Una neurona por cada clase

In [ ]:
# Compilar el modelo
model.compile(
    optimizer=Adam(learning_rate=0.001), # Optimizador
    loss='categorical_crossentropy', # Función de perdida
    metrics=['accuracy'] # Medir accuracy
)

In [ ]:
# Entrenamos el modelo
history = model.fit(
    train_generator,  # Generador de entrenamiento
    epochs=20,  # Número de épocas
    validation_data=val_generator,  # Generador de validación
    verbose=2
    )

# Visualización Resultados: Primera Red Neuronal

In [ ]:
plt.figure(figsize=(8, 5))

# Accuracy
plt.plot(history.history['accuracy'], label='Accuracy Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Accuracy Validación')

plt.title('Precisión durante el primer entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Valor')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))

# Loss
plt.plot(history.history['loss'], label='Loss Entrenamiento')
plt.plot(history.history['val_loss'], label='Loss Validación')

plt.title('Error durante el primer entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Valor')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Guardar todo el modelo (arquitectura + pesos + optimizer)
model.save("sign_language_model.h5")
print("Modelo guardado como sign_language_model.h5")

files.download("sign_language_model.h5")

#Metricas de Rendimiento: Primera Red Neuronal

In [ ]:
# Predicciones del conjunto de validación
y_prob = model.predict(val_generator, steps=len(val_generator), verbose=0)
y_pred = np.argmax(y_prob, axis=1) # Índice de la clase predicha
y_true = val_generator.classes # Clases reales
labels = list(val_generator.class_indices.keys()) # Nombres de las clases

# Reporte de clasificación
print("\nReporte de clasificación Primer Modelo:\n")
print(classification_report(y_true, y_pred, target_names=labels))

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_true, y_pred)

# Visualización
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=labels, yticklabels=labels)
plt.xlabel("Predicción")
plt.ylabel("Etiqueta real")
plt.title("Matriz de Confusión Primer Modelo")
plt.tight_layout()
plt.show()

# Creación de la Segunda Red Neuronal

Como los resultados del Primer Modelo no han sido los deseados, seguramente debido a la simplicidad de la arquitectura del mismo, ahora vamos a probar con un modelo más sofísticado.

Este nuevo modelo implementado corresponde a una arquitectura CNN profunda de tipo **VGG-like**, estructurada en varios bloques convolucionales seguidos de una capa densa para la clasificación, lo que permite una buena extracción de características y un rendimiento sólido en el reconocimiento de imágenes.

In [ ]:
model_cnn = Sequential()

# Bloque 1
model_cnn.add(Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(100,100,1)))
model_cnn.add(BatchNormalization())
model_cnn.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model_cnn.add(BatchNormalization())
model_cnn.add(MaxPooling2D((2,2)))
model_cnn.add(Dropout(0.25))

# Bloque 2
model_cnn.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model_cnn.add(BatchNormalization())
model_cnn.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model_cnn.add(BatchNormalization())
model_cnn.add(MaxPooling2D((2,2)))
model_cnn.add(Dropout(0.25))

# Bloque 3
model_cnn.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model_cnn.add(BatchNormalization())
model_cnn.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model_cnn.add(BatchNormalization())
model_cnn.add(MaxPooling2D((2,2)))
model_cnn.add(Dropout(0.30))

# Clasificación
model_cnn.add(Flatten())
model_cnn.add(Dense(256, activation='relu'))
model_cnn.add(Dropout(0.5))
model_cnn.add(Dense(10, activation='softmax'))

model_cnn.compile(optimizer=Adam(learning_rate=0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Mejoras de este Nuevo Modelo con respecto al Primer Modelo:

- Modelo mucho más profundo → con más capas convolucionales y más filtros el modelo puede aprender patrones más complejos (dedos, forma de la mano) y distinguir clases muy similares.

- BatchNormalization → normaliza la salida de cada capa para que el entrenamiento sea más estable y rápido.

- Dropout al final de cada bloque → apaga aleatoriamente neuronas durante el entrenamiento, evitando que el modelo memorice posturas concretas y falle cuando cambia la mano.

- Data augmentation fuerte → genera múltiples versiones de cada imagen, simulando variaciones reales provocando que el modelo se vuelva resistente a cambios de ángulo, escala o iluminación.

- Clasificador final más potente → Flatten + Dense 256 combina todas las características extraídas por las convoluciones dandole al modelo la capacidad para discriminar mejor entre clases visualmente parecidas.

- Learning rate más bajo (0.0005) → evita que la red se pase del mínimo local y ayuda a afinar detalles, importante con imágenes difíciles.

- Implementacion de Callbacks: Se añadieron varios callbacks que optimizan el proceso de entrenamiento haciendo que este sea más lo más estable, más corto, y nos dé el modelo con mejor capacidad de generalización.

In [ ]:
# --- Generador de entrenamiento con data augmentation ---
train_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,      # rota las imágenes hasta ±15 grados
    width_shift_range=0.10, # desplaza horizontalmente hasta 10%
    height_shift_range=0.10,# desplaza verticalmente hasta 10%
    zoom_range=0.10,        # aplica zoom aleatorio hasta ±10%
    shear_range=0.10        # aplica deformación tipo “cizalla” hasta 10%
).flow_from_dataframe(
    df_train,
    x_col='filename',
    y_col='class',
    target_size=(100,100),
    color_mode='grayscale',
    batch_size=20,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

Callbacks de la Segunda Red Neuronal:

- EarlyStopping → detiene el entrenamiento automáticamente si el modelo deja de mejorar en validación, evitando sobreajuste y ahorrando tiempo.

- ReduceLROnPlateau → si el modelo se estanca, reduce la tasa de aprendizaje a la mitad, lo que permite afinar la red y mejorar precisión en clases difíciles.

- ModelCheckpoint → guarda el mejor modelo durante el entrenamiento para no perder los pesos óptimos.

In [ ]:
# --- Callbacks ---
callbacks = [
    # Detiene el entrenamiento si la pérdida de validación no mejora durante 6 épocas
    EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, verbose=1),

    # Reduce la tasa de aprendizaje si la pérdida de validación se estanca durante 4 épocas
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-7, verbose=1),

    # Guarda automáticamente el modelo con menor pérdida de validación
    ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)
]

# Entrenamiento Segunda Red Neuronal

In [ ]:
uploaded = files.upload()

In [ ]:
# Carga los pesos guardado del entrenamiento
model_cnn.load_weights("model_cnn_bloque2_20.weights.h5")

En este caso este Nuevo Modelo lo entrenaremos con un Epochs=28

In [ ]:
# --- Entrenamiento ---
history = model_cnn.fit(
    train_generator,
    epochs=28,              # máximo número de épocas
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=2
)

In [ ]:
# Guardar PESOS del Modelo
model_cnn.save_weights("model_cnn_numeros.weights.h5")
print("Pesos guardados como model_cnn_numeros.weights.h5")

files.download("model_cnn_numeros.weights.h5")

# Evaluación en Test

In [ ]:
test_loss, test_acc = model_cnn.evaluate(test_generator, verbose=2)
print(f"Precisión en test: {test_acc:.2f}")

In [ ]:
# Predicciones del conjunto de test
y_prob = model_cnn.predict(test_generator, steps=len(test_generator), verbose=0)
y_pred = np.argmax(y_prob, axis=1)  # Índice de la clase predicha
y_true = test_generator.classes     # Clases reales
labels = list(test_generator.class_indices.keys())  # Nombres de las clases

# Reporte de clasificación
print("\nReporte de clasificación Segundo Modelo:\n")
print(classification_report(y_true, y_pred, target_names=labels))

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_true, y_pred)

# Visualización
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=labels, yticklabels=labels)
plt.xlabel("Predicción")
plt.ylabel("Etiqueta real")
plt.title("Matriz de Confusión Segundo Modelo")
plt.tight_layout()
plt.show()

# Serialización del Modelo

In [ ]:
# Guardar TODO el Modelo (arquitectura + pesos + optimizer)
model_cnn.save("model_cnn_numeros.h5")
print("Modelo guardado como model_cnn_numeros.h5")

files.download("model_cnn_numeros.h5")

# Cargar el Modelo

In [ ]:
url = 'https://drive.google.com/uc?id=1JmfrhsnctvXNG4DI0Ik8QuKA0JG5Ay7X'
output = "model_cnn_cargado.h5"
gdown.download(url, output, quiet=False)
model_cnn = load_model(output)
print("Modelo cargado correctamente.")

In [ ]:
test_loss, test_acc = model_cnn.evaluate(test_generator, verbose=2)
print(f"Precisión en test: {test_acc:.2f}")